# Westeros Tutorial - Implementation of cooling technologies



In [1]:
import pandas as pd
import ixmp
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In /Users/MuhammadAwais/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/MuhammadAwais/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/MuhammadAwais/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
mp = ixmp.Platform(name = 'local')

In [3]:
model = 'Westeros Electrified'

base = message_ix.Scenario(mp, model=model, scenario='baseline')
scen = base.clone(model, 'cooling tech','llustrating cooling tech using addon',
                  keep_solution=False)
scen.check_out()

In [4]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'

## Retrieve parameters to perform subsequent addition of parameters

In [5]:
#year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Westeros'
gdp_profile = pd.Series([1., 1.5, 1.9], index=pd.Index([700, 710, 720], name='Time'))

## Defining input for different parameters 

We add `water supply` as new level of water supply and `freshwater_supply` & `saline_supply` as commodities

In [6]:
scen.add_set("commodity", ["freshwater_supply", "saline_supply","cooling_req","freshwater_instream"])
scen.add_set("level", ["water_supply","cooling"])

In [7]:
base_input = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'node_origin': country,
    'commodity': 'electricity',
    'time': 'year',
    'time_origin': 'year',
}

base_output = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'node_dest': country,
    'time': 'year',
    'time_dest': 'year', 
    'unit': '%',
}


base_capacity_factor = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'time': 'year',
    'unit': '%',
}

base_technical_lifetime = {
    'node_loc': country,
    'year_vtg': model_horizon,
    'unit': 'y',
}

base_inv_cost = {
    'node_loc': country,
    'year_vtg': model_horizon,
    'unit': 'USD/GWa',
}

base_fix_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'unit': 'USD/GWa',
}

base_var_cost = {
    'node_loc': country,
    'year_vtg': vintage_years,
    'year_act': act_years,
    'mode': 'standard',
    'time': 'year',
    'unit': 'USD/GWa',
}

### addon technology - interlinking technologies
In order to demonstrate the use of this feature, we will be adding a new demand for water to this scenario. This demand will be met by the coal_ppl via three cooling technologies, the addon technology.

We will therefore go through the following steps:

#### Cooling technology types
Add new technologies:
1. Once-through cooling by freshwater - `coal_ppl_ot_fresh`
2. Closed loop cooling by freshwater - `coal_ppl_cl_fresh`
3. Once through cooling seawater `coal_ppl_ot_saline`
4. Air cooling `coal_ppl_air`

Link these cooling tehcnologies to the `coal_ppl` using the addon feature.

## Adding freshwater supply & instream extraction as techs

The water supply technologies have only input & output dataframe in model. 

*At some point, we may want to link it to global water supply model in future


In [8]:
tec = 'extract_freshwater_supply'

scen.add_set('technology', tec)

ext_fresh_water = make_df(base_output, technology=tec, commodity='freshwater_supply', 
                   level='water_supply', value=1.0)

scen.add_par('output', ext_fresh_water)

tec = 'extract__freshwater_instream'

scen.add_set('technology', tec)

ext_freswater_instream  = make_df(base_output, technology=tec, commodity='freshwater_instream', 
                   level='water_supply', value=1.0)

scen.add_par('output', ext_freswater_instream)



### Now we add parameters for cooling technologies

We will be using `water required per heat output` for cooling technologies as input value of cooling technology types. 


In [9]:
tec = 'ct_ot_fresh' 
scen.add_set('technology', tec)

out = make_df(base_output, technology=tec, commodity='cooling_req', 
                   level='cooling', value=1.0)

scen.add_par('output', out )


inp = make_df(base_input, technology=tec, commodity='freshwater_supply',  
                  level='water_supply', value=0.938, unit='-')

scen.add_par('input', inp)


invc = make_df(base_inv_cost, technology=tec, value=60)
scen.add_par('inv_cost', invc)

fixc = make_df(base_fix_cost, technology=tec, value=0)
scen.add_par('fix_cost', fixc)

tl = make_df(base_technical_lifetime, technology=tec, value=30)
scen.add_par('technical_lifetime', tl)


tec = 'ct_cl_fresh' 

scen.add_set('technology', tec)

out = make_df(base_output, technology=tec, commodity='cooling_req', 
                   level='cooling', value=1.0)

scen.add_par('output', out)

# Since closed loop fresh uses electricity and freshwater as input, so we use both

inp = make_df(base_input, technology=tec, commodity='electricity',  
                  level='secondary', value=0.017, unit='-')

inp1 = make_df(base_input, technology=tec, commodity='freshwater_supply',  
                  level='water_supply', value=0.017, unit='-')


scen.add_par('input', inp)
scen.add_par('input', inp1)


invc = make_df(base_inv_cost, technology=tec, value=700)
scen.add_par('inv_cost', invc)

fixc = make_df(base_fix_cost, technology=tec, value=0)
scen.add_par('fix_cost', fixc)

tl = make_df(base_technical_lifetime, technology=tec, value=30)
scen.add_par('technical_lifetime', tl)

tec = 'ct_ot_saline' 

scen.add_set('technology', tec)

out = make_df(base_output, technology=tec, commodity='cooling_req', 
                   level='cooling', value=1.0)

scen.add_par('output', out)


inp = make_df(base_input, technology=tec, commodity='saline_supply',  
                  level='water_supply', value=0.938, unit='-')

scen.add_par('input', inp)


invc = make_df(base_inv_cost, technology=tec, value=200)
scen.add_par('inv_cost', invc)

fixc = make_df(base_fix_cost, technology=tec, value=0)
scen.add_par('fix_cost', fixc)

tl = make_df(base_technical_lifetime, technology=tec, value=30)
scen.add_par('technical_lifetime', tl)

tec = 'ct_air' 

scen.add_set('technology', tec)

out = make_df(base_output, technology=tec, commodity='cooling_req', 
                   level='cooling', value=1.0)

scen.add_par('output', out)


inp = make_df(base_input, technology=tec, commodity='electricity',  
                  level='secondary', value=0.102, unit='-')

scen.add_par('input', inp)


invc = make_df(base_inv_cost, technology=tec, value=530)
scen.add_par('inv_cost', invc)

fixc = make_df(base_fix_cost, technology=tec, value=0)
scen.add_par('fix_cost', fixc)

tl = make_df(base_technical_lifetime, technology=tec, value=30)
scen.add_par('technical_lifetime', tl)

## Addon mapping 
We try to define a single `type_addon` for each power plant. Here we use `coal_ppl` as an example. 

`type_addon` refers to cooling technologies for `coal_ppl` ,whereas `addon` refers to cooling technology types. 

In [23]:
# If we need to add more power plants, we will be using more type_addon here and use the heat rate or cooling requirement per unit of output here
type_addon = 'cool_coal_ppl'


# Adding cooling technology types 
addon = ('ct_ot_fresh','ct_cl_fresh','ct_ot_saline','ct_air')

scen.add_set('addon', pd.DataFrame({'technology': addon})) 

# Parent technology to map 
tec = 'coal_ppl'

# Mapping addon with type_addon and parent tech
scen.add_cat('addon', type_addon, addon)
scen.add_set('map_tec_addon', pd.DataFrame({'technology': tec,
                                            'type_addon': [type_addon]}))

In this case, we use addon conversion factor for each `type_addon`.Addon conversion factor refers to the heat produced by the parent technology (in this case `coal_ppl`) that is needed to be cooled. 
For now for simplicity, we are referring to heat rate which is 1/efficiency. In the current scenario would be 5. 

In [24]:
df = pd.DataFrame({'node': country,
                   'technology': tec,
                   'year_vtg': vintage_years,
                   'year_act': act_years,
                   'mode': 'standard',
                   'time': 'year',
                   'type_addon': type_addon,
                   'value': 5,
                   'unit': '%'})

scen.add_par('addon_conversion', df)

Now we bound the activity of `cool_coal_ppl` to 100% share of `coal_ppl` to make sure that all of the heat produced is being cooled.

In [25]:
df = pd.DataFrame({'node': country,
                   'technology': tec,
                   'year_act': act_years,
                   'mode': 'standard',
                   'time': 'year',
                   'type_addon': type_addon,
                   'value': 1,
                   'unit': '%'})
scen.add_par('addon_lo', df)

## Commit & Solve 

In [26]:
scen.commit(comment='implement cooling tech')
scen.set_as_default()

In [27]:
scen.solve()

In [28]:
scen.var('OBJ')['lvl']

256945.484375

In [30]:
scen.var('ACT')

,node_loc,technology,year_vtg,year_act,mode,time,lvl,mrg
0,Westeros,coal_ppl,690,700,standard,year,15.525114,0.000000e+00
1,Westeros,coal_ppl,700,700,standard,year,65.011181,0.000000e+00
2,Westeros,coal_ppl,700,710,standard,year,65.011181,0.000000e+00
3,Westeros,coal_ppl,710,710,standard,year,81.430746,0.000000e+00
4,Westeros,coal_ppl,710,720,standard,year,81.430746,0.000000e+00
5,Westeros,coal_ppl,720,720,standard,year,129.680365,0.000000e+00
6,Westeros,wind_ppl,690,700,standard,year,10.350076,0.000000e+00
7,Westeros,wind_ppl,700,700,standard,year,20.224739,0.000000e+00
8,Westeros,wind_ppl,700,710,standard,year,20.224739,0.000000e+00
9,Westeros,wind_ppl,710,710,standard,year,0.000000,0.000000e+00


In [ ]:
mp.close_db()